<a href="https://colab.research.google.com/github/jukdang/AI_repo/blob/main/Dacon_toursism%20data%20classification/travel4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
cd ./drive/MyDrive/

[Errno 2] No such file or directory: './drive/MyDrive/'
/content/drive/MyDrive


In [28]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, IterableDataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

In [29]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup

In [31]:
import torchvision
from torchvision import transforms

In [32]:
!pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import os

device = torch.device("cuda:0")

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

0


In [34]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [35]:
from sklearn.model_selection import train_test_split

In [36]:
train_df = pd.read_csv('train_processed2.csv')

In [37]:
train_df

,id,resized_img_path,overview,sentence_length,category1,category2,category3,tokenized_overview
0,TRAIN_00000,./image/resized_train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,217,0,0,0,"['소안항', '은', '조용한', '섬', '으로', '인근', '해안이', '청..."
1,TRAIN_00001,./image/resized_train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,244,1,1,1,"['경기도', '이천시', '모가면', '에', '있는', '골프장', '으로', ..."
2,TRAIN_00002,./image/resized_train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,160,2,2,2,"['금오산성', '숯불', '갈비', '는', '한우', '고기', '만을', '전..."
3,TRAIN_00003,./image/resized_train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,67,2,2,2,"['철판', '위', '에서', '요리', '하는', '안동찜닭', '을', '맛볼..."
4,TRAIN_00004,./image/resized_train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,63,2,2,2,"['※', '영업', '시간', '10:30', '~', '20:30', '\n\n..."
...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/resized_train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,50,2,2,2,"['해발', '12000', 'm', '에', '자리', '한', '식담', '겸'..."
16982,TRAIN_16982,./image/resized_train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,471,4,7,11,"['설악힐', '호텔', '은', '동해', '고속도로', '속초', '톨게이트',..."
16983,TRAIN_16983,./image/resized_train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,282,4,7,11,"['충남', '서산시', '중심', '가에', '위치', '한', '줌', '모텔'..."
16984,TRAIN_16984,./image/resized_train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,158,1,1,3,"['토', '토큰', '바위', '캠핑장', '은', '경기도', '가평', '지역..."


In [38]:
train_df['drive_img_path'] = './' + train_df['resized_img_path'].str[8:]

In [39]:
catsize = len(train_df['category3'].unique())
print(catsize)

128


In [40]:
train, test = train_test_split(train_df)
train_df = train[['drive_img_path','category1']].to_numpy()
test_df = test[['drive_img_path','category1']].to_numpy()

In [41]:
torchvision.io.read_image(train_df[0][0]).type(torch.float64)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], dtype=torch.float64)

In [42]:
class EfficientNetDataset(IterableDataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.transform = transforms.Compose([transforms.Resize((224,224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

    def __iter__(self):
      for path, label in self.dataset:
        img = torchvision.io.read_image(path).type(torch.float32)
        img = trans(img)
        label = np.int32(label)
        yield (img, label)

In [43]:
# Setting parameters 필수
batch_size = 16
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [44]:
data_train = EfficientNetDataset(train_df)
data_test = EfficientNetDataset(test_df)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [45]:
class EfficientNetClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 eff,
                 hidden_size = 1000,
                 num_classes=catsize,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(EfficientNetClassifier, self).__init__()
        self.eff = eff
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def forward(self, image):
        
        out = self.eff(image)
        if self.dr_rate:
            out = self.dropout(out)
        out = self.classifier(out)
        return out

In [46]:
from efficientnet_pytorch import EfficientNet
efficientnet = EfficientNet.from_pretrained('efficientnet-b4')

Loaded pretrained weights for efficientnet-b4


In [47]:
trans = transforms.Compose([transforms.Resize(224),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
test = torchvision.io.read_image(train_df[0][0]).type(torch.float32).unsqueeze(0)
input = trans(test)
print(input.shape)
out = efficientnet(input)
print(out.shape)

torch.Size([1, 3, 224, 336])
torch.Size([1, 1000])


In [48]:
model = EfficientNetClassifier(efficientnet,  dr_rate=0.5).to(device)
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_df) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (img, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        img = img.float().to(device)
        label = label.long().to(device)

        out = model(img)

        loss = loss_fn(out, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (img, label) in enumerate(tqdm_notebook(test_dataloader)):
        img = img.float().to(device)
        label = label.long().to(device)
        out = model(img)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

epoch 1 batch id 1 loss 4.9510087966918945 train acc 0.0


KeyboardInterrupt: ignored